In [1]:
import pyspark
from pyspark.sql import SparkSession
import pandas as pd

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

23/03/06 16:07:47 WARN Utils: Your hostname, wedivv-H110M-S2V resolves to a loopback address: 127.0.1.1; using 192.168.1.44 instead (on interface wlp5s0)
23/03/06 16:07:47 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/06 16:07:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [17]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [18]:
df_green.createOrReplaceTempView('green')

In [23]:
df_green_revenue = spark.sql("""
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2

""")

In [25]:
df_green_revenue.show()

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2020-01-10 16:00:00|  66|235.20000000000005|            12|
|2020-01-12 09:00:00| 108|            134.12|             5|
|2020-01-28 22:00:00| 223|            123.08|            17|
|2020-01-22 11:00:00| 222|            197.63|             6|
|2020-01-19 16:00:00| 205|             70.47|             3|
|2020-01-05 22:00:00| 177|             34.11|             1|
|2020-01-17 16:00:00|  33| 505.8300000000002|            32|
|2020-01-19 22:00:00|  41| 537.6600000000002|            39|
|2020-01-11 16:00:00|  41| 995.4999999999992|            65|
|2020-01-03 11:00:00| 250|             120.1|             7|
|2020-01-26 13:00:00| 210|             41.41|             2|
|2020-01-10 09:00:00| 179|180.78999999999996|             9|
|2020-01-30 06:00:00| 168|             84.94|             3|
|2020-01-22 08:00:00|  7

In [26]:
df_green_revenue.write.parquet('data/report/revenue/green', mode='overwrite')

In [27]:
df_yellow = spark.read.parquet('data/pq/yellow/*/*')

In [28]:
df_yellow.createOrReplaceTempView('yellow')

In [31]:
df_yellow_revenue = spark.sql("""
SELECT 
    date_trunc('hour', tpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    yellow
WHERE
    tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2

""")

In [33]:
df_yellow_revenue.write.parquet('data/report/revenue/yellow', mode='overwrite')

In [37]:
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_records')

df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_number_records')

In [38]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'zone'], how='outer')

In [39]:
df_join.show()

+-------------------+----+------------------+--------------------+------------------+---------------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+----+------------------+--------------------+------------------+---------------------+
|2020-01-01 00:00:00|  22|              15.8|                   1|              null|                 null|
|2020-01-01 00:00:00|  29|              61.3|                   1|              null|                 null|
|2020-01-01 00:00:00|  35|            129.96|                   5|              null|                 null|
|2020-01-01 00:00:00|  41|1363.9599999999984|                  84|           1256.53|                   80|
|2020-01-01 00:00:00|  55|129.29000000000002|                   4|              null|                 null|
|2020-01-01 00:00:00|  65|199.48999999999998|                  10| 409.3500000000001|                   19|
|2020-01-01 00:00:00|  72|  

In [40]:
df_join.write.parquet('data/report/revenue/total')

In [42]:
df_join = spark.read.parquet('data/report/revenue/total/')

In [43]:
df_join.show()

+-------------------+----+------------------+--------------------+------------------+---------------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+----+------------------+--------------------+------------------+---------------------+
|2020-01-01 00:00:00|   3|              null|                null|              25.0|                    1|
|2020-01-01 00:00:00|   4|              null|                null|1004.3000000000001|                   57|
|2020-01-01 00:00:00|   7| 769.7299999999996|                  45| 455.1700000000002|                   38|
|2020-01-01 00:00:00|  17|            195.03|                   9|220.20999999999998|                    8|
|2020-01-01 00:00:00|  18|               7.8|                   1|               5.8|                    1|
|2020-01-01 00:00:00|  25|             531.0|                  26|324.34999999999997|                   16|
|2020-01-01 00:00:00|  36|29

In [44]:
df_zones = spark.read.parquet('zones/')

In [45]:
df_zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [46]:
df_result = df_join.join(df_zones, df_join.zone == df_zones.LocationID)

In [51]:
df_result.drop('LocationID', 'zone').write.parquet('tmp/revenue-zones')